#### In order to generate a Eulirian Path from DNA Read Pairs, an artificial Eulirian Cycle will  be created first. 



## Eulirian Cycle Concept And Application for Sequencing DNA from Read_Pairs


In a directed and balanced graph, read_pairs are the imaginary edges and their prefixes & suffixes are nodes,
submitted in a dict as keys and values, respectively. An example of read_pairs is  (TAA,GCC) where comma indicates a 
specific distance between reads/kmers. The nodes (prefixes & suffixes) that are derived from this read_pair edge are :
                                                
                                                (TA,GC)->(AA,CC)


All edges must be entered once.
Each time a value of a dict is entered, an edge is used, since the value is called from a the dict's key.
As mentioned above, the key along with the value constitutes the kmer, which is an edge.
        
1) First, select randomly a) a kmer , b) (one of) that kmer's value(s).
Append the kmer and its value to a list. 
From now on, only append chosen values through next steps. Do not append keys.
           
2) Move to another vertex by using that value as dict key and calling (one of) it value(s). Repeat the process.
Save the first dict key with more than one outdegrees (efferent edges) a potential new start node that might be used later on. 

3) When another vertex is not accessible anymore, that is, the last value used corresponds to a dict key that 
has been emptied, a cycle has been completed. Check if all values have been appended to the list, that is, check if eulirian cycle has been completed (while condition). 
           
4) If cycle is not complete (not Eulirian), transform the list with the appended values according to this :
        
    start node,2nd node,3rd node,potential exit node, 5th node,6th node, 7th node
                                                   ||
                                                   \/
  potential exit node, 5th node,6th node, 7th node, ~~startnode~~ ,2nd node,3rd node,potential exit node.
           
This method is valid since incomplete cycles in directed & balanced graphs end up solely at starting points. 
Hence, the last node (7th) leads back to the starting node. For this reason, all nodes(vertices) starting from
the potential exit node up to the last node can be inserted at the start of the list. This way, a new cycle is
created, where the potential exit node has now become the new start node AND the last node(so far).
The potential exit node has still unused efferents (outdegrees) and therefore helps to continue searching for 
a Eulirian Cycle. 
           
Be aware of the ~~start node~~. The start node, that is the very first dict value, acts as dict key
and keys shouldn't exist anywhere else except for the start of the cycle. Yet, one more transformation
will PUSH it forward and render it inside the cycle. Hence, the previous start node must NOT be 
incorporated in a new cycle during the transformation of the latter. However, a new start node always exists,
since the new transformed cycle will start with a new node which represents a dict key.

In [ ]:
def Eulirian_Path (adjacency_dic):
    """ Input : Dic with tuple as key (2 kmers' prefixes) and LIST that contain tuple(s) (same kmers' suffixes) as value.
                This is the output of DeBruinj_for_Read_Pairs(). For a better conceptualization of nodes, please refer to
                the description of DeBruijn_for_Read_Pairs.
                
        Output: list with int or str as elements. Elements in list indicate nodes and must be as many as the input's dict values() 
                + 1 element that is inserted at the start to indicate a key (start) node in contrast to the rest which are all dict values.
                                                                                                                        
                From a graph (input), checks whether a Eulirian Path exists. If so, a Eulirian cycle is first created by joining the
                edges (extreme nodes and NOT graph edges) of the graph. Then, the Eulirian Path is found by rotating the final Eulirian Cycle accordingly
 
    
                                                                                                                        """
    import random
    from collections import deque
    
    N_of_nodes   = sum (list(map(lambda x: len(x), adjacency_dic.values())))             # dict values are lists, thus measure the elements inside them (nodes). x is the list and len(x) the number of its elements which are tuples           
    first_key,last_key = Eulirian_Edges_for_Read_Pairs (adjacency_dic)                   # These should be the final first and last nodes
    adjacency_dic = Path_With_Joined_Edges_for_Read_Pairs (adjacency_dic,first_key,last_key) # Create an artificial cycle by joining first and last node
    cycle        = deque([first_key, adjacency_dic[first_key].pop()])                    # first dict value element is accessed via first key, first edge is used.
    exit         = 0                                                                     # distance of first exit node after a cycle, from the end of the next cycle 
    start_key    = cycle[0]                                                              # the only element of cycle that represents key instead of value. Always in start.
      
    while len(cycle) != N_of_nodes +1 :                                                  #+1 because Eulirian cycle's length = a start key node in start + dict values' elements 
        current_node = cycle[-1]                                                         

        if len(adjacency_dic[current_node]) > 1 and exit == 0 :                          # at least two elements in dic value (list) and no exit node found yet. exit == 0 to prevent from changing exit_nodes when new ones are entered
            exit_node = cycle[-1]                                                        ## only the first exit node is needed in every new cycle for wayout
        
        if adjacency_dic[current_node] != [] :                                           # at least one element in dic value(list)
            current_node = adjacency_dic[current_node].pop()                             
            cycle.append(current_node)
            try:
                exit_node
                exit +=1                                                                # first exit node's distance from end of cycle increases with new elements
            except:                                                                     # without an exit node present, don't measure distance yet
                pass      

        else :                                                                          #dead-end, no value left for this node
            cycle.remove(start_key)                                                     #remove the start key from cycle. This will also facilitate rotating exit node to the start instead of end of cycle
            cycle.rotate(exit)                                                          #move exit node to the start of the cycle and maintain elements' sequence
            cycle.append (cycle[0])                                                     #append the exit node at the end of the cycle to complete a cycle and continue from there
            start_key = cycle[0]                                                        #save the start key for the next dead-end
            exit = 0                                                                    #exit=0 so that we can find the first exit node and save it                                                               
    
    while cycle[-1] != last_key:                                                        #as long as the last key is not at last position
        cycle.rotate(-cycle.index(first_key))                                           #move all the nodes to the left until a first key occurence is at first place
  
    return cycle

In [ ]:
def Eulirian_Edges_for_Read_Pairs (adjacency_dic):
    """Input : Dic with tuple as key (kmers' prefixes) and LIST that contain tuple(s) (same kmers' suffixes) as value.
       Output :list with two tuples. Each tuple contains two str elements. 
       The first tuple corresponds to the first node (based on read_pairs) of a Eulirian Path.
       The second tuple corresponds to the last node (based on read_pairs) of a Eulirian Path.
       
       Checks whether a Eulirian path exists (2 unbalanced nodes and a graph which is connected). If so, finds the endpoint nodes.
       Note that an 'edge' here indicates that these nodes are at the endpoints of the path. Thus, it doesn't indicate graph edges"""
    
    
    outdegrees_indegrees_sum = dict.fromkeys(adjacency_dic, 0)
    more_afferents = []                                                   # nodes with more indegrees than outdegrees
    more_efferents = []                                                   # nodes with more outdegrees than outdegrees
    
    
    for key, values in adjacency_dic.items():                              
        outdegrees_indegrees_sum[key] += len(values)                      # add N of outdegrees for each key-node
        
    for key,values in adjacency_dic.items():                               
        for node in values:
            try :
                outdegrees_indegrees_sum[node] -= 1                       # remove 1 every time a node is an indegree
            except:
                outdegrees_indegrees_sum[node] = -1                       #create artificial node

    for key,value in outdegrees_indegrees_sum.items():            
        if value > 0:
            more_efferents.append([key,value])                            # append tuple and value to a list - > [('',''),value]
        if value < 0:
            more_afferents.append([key,value])
        else:
            pass
    
            
    if len(more_efferents)==1 and len(more_afferents) == 1  :              #2 unbalanced node condition for Eulirian Path. len == 1 when only one list has been appended
            if abs (more_efferents[0][1]) == abs (more_afferents[0][1]) :  #if the values are equal 
                return [more_efferents[0][0],more_afferents[0][0]]         #return the keys/nodes      
    
    else :
        print('There are not 2 almost balanced edges that can support Eulirian Path afterwards')

In [ ]:
def Path_With_Joined_Edges_for_Read_Pairs (adjacency_dic, first_key, last_key):
    """Input 1: Dic with tuple as key (kmers' prefixes) and LIST that contain tuple(s) as value (same kmers' suffixes).
                The key and its value(s) are adjacent nodes/vertices that together form a read_pair
       Input 2: tuple with 2 str elements. The tuple corresponds to the first node (based on read_pairs) of a Eulirian Path.
       Input 3: tuple with 2 str elements. The tuple corresponds to the last node  (based on read_pairs) of a Eulirian Path.
       Output : Dic with tuple as key (kmers' prefixes) and LIST that contain tuple(s) (same kmers' suffixes) as values. 
                The key and its value(s) [tuple or tuples with kmers/reads] are adjacent nodes that together form a read_pair.
       
       In a dictionary from which a Eulirian Path can be formed, join the first and last unbalanced node to create a Eulirian cycle """
        
    if last_key in adjacency_dic.keys():
        adjacency_dic[last_key].append(first_key)
    else:
        adjacency_dic[last_key] = [first_key]
        
    return adjacency_dic


In [ ]:
def DeBruijn_for_Read_Pairs (read_pairs):
    """Input : list with tuples that contain (k,d)-mers. That is a pair of kmers of length k and distance d between them.
       Output : Dic with tuple as key (kmers' prefixes) and LIST that contain tuple(s) (same kmers' suffixes) as values.
       
       For each read_pair, create a tuple with the prefixes of its reads and a value with the suffixes of its reads.
       Read_pairs are (k,d)-mers like this : 
                                               TAA|GCC  
       
       The symbol | indicates distance and replaces an unknown base. It would be || for distance = 2 and so on.
       The symbol | is for illustrative reasons. Regardless the distance, the latter is merely indicated by the "," of the input tuples) 
       The prefixes of a read_pair's reads are read1[:-1] & read2[:-1] and the suffixes are read1[1:] and read2[1:].
       Hence, the read_pair TAA|GCC has (TA,GC) as a prefix and (AA,CC) as a suffix.
       If a read_pair appears multiple times, append a tuple with its suffixes as value for an equal number of times.
       For instance, if there are two  TAA|GCC  in the input, the respective part of output will be : 
                                
                                (TA,GC)-> [(AA,CC),(AA,CC)]
                                      
                                    or, equivalently
                
                            dic[(TA,GC)]-> [(AA,CC),(AA,CC)]
       
       Note that when the read_pairs can form a Eulirian path, each key/vertex/node is not only a pair of suffixes of a read_pair 
       but also a pair of prefixes of another key (apart from start & end in RIGHT sequence). 
       DeBruijn's output can be used to 'glue' read_pairs afterwards. """
    
    adjacency_dic = {}
    
    for pair in read_pairs:                                   # pair's type is list, elements are tuples that contain str
        prefixes = (pair[0][:-1],pair[1][:-1])
        suffixes = (pair[0][1:],pair[1][1:])
        
        if prefixes not in adjacency_dic:
            adjacency_dic[prefixes] = []
        else:
            pass
        
        adjacency_dic[prefixes].append(suffixes)
    
    return adjacency_dic

In [ ]:
def Read_Pair_Path_to_Genome (read_pairs):
    """Input : list with tuples that constitute read_pairs of this form : (ACC,GCC). 
               The first element of tuple is the suffix and the second the prefix. Kmers are separated by a distance d which is 
               occupied by an unknown base. Think of the "," in (ACC,GCC) as a space with unknown bases between ACC and GCC.
       Output: list with 2 strings that corresponds to the final paths of suffices and prefices, respectively.
       
       An already PROPERLY arranged sequence of read_pairs is submitted as input and the final genome for suffixes and prefixes is created."""
    
    suffices = []
    prefices = []
    
    for pair in read_pairs:
        suffices.append(pair[0])
        prefices.append(pair[1])
        
    sfx_path = suffices.pop(0)
    for sfx in range(len(suffices)):
        sfx_path += suffices[sfx][-1]
        
    prfx_path = prefices.pop(0)
    for prfx in range(len(prefices)):
        prfx_path += prefices[prfx][-1]
        
    return [sfx_path,prfx_path]

In [ ]:
def Overlap_in_Read_Pairs (suffix_path, prefix_path,kmer_length,distance):
    """Input 1: string
       Input 2: string
       Input 3: int indicating the length of one read(kmer) in read_pairs before the paths were created
       Input 4: int indicating the distance between read_pairs before the paths were created
       Output : False or nothing
       
       From an upper path of kmers and a lower path of kmers derived from a graph comprising nodes withupper and lower kmers, 
       checks whether the overlapping characters are identical.
       For instance, for paths created by (2,1)mers with length 2 and distance 1:
                        
                        upper path -> A G C A G C T G C T 
                        lower path ->       A G C T G C T G C A
                        
      The overlapping characters are identical, thus nothing will be returned. Otherwise, False would be returned. """
    
    comparable_sfx_path = suffix_path[kmer_length+distance:]
    comparable_prfx_path= prefix_path[:len(prefix_path) - (kmer_length+distance) ]
    
    for index in range(len(comparable_prfx_path)):
        if comparable_sfx_path[index] == prefix_path[index]:
            pass
        else :
            return False
            break 

###### assemble final path from upper and lower path derived from read_pairs
(upper and lower paths are Eulirian Paths derived from the same nodes. Each DeBruijn node is contains two kmers/reads which are prefixes of a read pair and suffixes of another (except the 2 endpoint nodes, unless these endpoints have been artificially joined to form a Eulirian Cycle) 

In [ ]:
### Putting it all together

kmer_length = x
distance    = y

debruijn_graph = DeBruijn_for_Read_Pairs(some_reads_of_yours)
path = Eulirian_Path(debruijn_graph)                                                # Path WITH nodes containing BOTH prefixes and suffixes is created here
suffix_path, prefix_path = Read_Pair_Path_to_Genome(path)                           # Prefixes and suffixes are separated and a path without nodes is created for each

if Overlap_in_Read_Pairs(suffix_path,prefix_path,kmer_length,distance) != False:    #A final path might be created without corresponding to a real path
                                                                                    ## To make sure that the final path corresponds to a real one, 
                                                                                    ### an identical match of overlapping bases must exist. Like below
    
    final_path = suffix_path+prefix_path[len(suffix_path)-(kmer_length+distance):]  #Assemble final path upper path -> A G C A G C T G C T 
                                                                                                        #lower path ->        A G C T G C T G C A
                                                                                                     #final path ->  A G C A G C T G C T G C A